**# Masoud Karami**      **2050716**


[INF8953CE(Fall 2020) : Machine Learning](http://sarathchandar.in/teaching/ml/fall2020/)
[Assignment #2](https://drive.google.com/file/d/10TncIjTPBt0kkIEm8XlFdDcJrd4Bopse/view) Due on : Oct 26, 10:00 pm

* You have to submit the pdf copy of the report on gradescope before the deadline. If you
handwrite your solutions, you need to scan the pages, merge them to a single pdf file and
submit. Mark page 1 for outline items $Late\ Submission$ and $Verbosity$.

* When asked to report the parameters, save the corresponding parameters in a text file
(.txt) with the following template for the name: 
`Assignment1 <Matricule> <section> <sub-question > <sub-sub-question>`. Submit all the text files and codes compressed
to a single file `<your-matricule>.zip` on Moodle.

$Note:$ gradescope doesn’t accept `.zip`.

* Be precise with your explanations in the report. Unnecessary verbosity will be penalized.
* You are free to use libraries with general utilities, such as `matplotlib`, `numpy` and `scipy` for
`python`. However, you should implement the algorithms yourself, which means **you should not** use pre-existing implementations of the algorithms as found in `SciKit learn`,
Tensorflow, etc.!

---
---

# Linear Classification and Nearest Neighbor Classification

> **1.** You will use a synthetic data set for the classification task that you’ll generate yourself.
Generate two classes with $20$ features each. Each class is given by a multivariate Gaussian
distribution, with both classes sharing the same covariance matrix. You are provided
with the mean vectors ($DS1-m0$ for mean vector of negative class and $DS1-m1$ for mean
vector of positive class) and the covariance matrix ($DS1-cov$). Generate $2000$ examples
for each class, and label the data to be positive if they came from the Gaussian with
mean m1 and negative if they came from the Gaussian with mean $m0$. Randomly pick
(without replacement) $20%$ of each class (i.e., $400$ data points per class) as test set, $20%$
of each class (i.e., $400$ data points per class) as validation set and train the classifiers
on the remaining $60%$ data. When you report performance results, it should be on the
test set. Call this dataset as $DS1$, and submit it with your code. Follow the instructions
from Assignment 1 for data submission format.

In [ ]:
# import sys
# import os


# filelist = [ f for f in os.listdir('/content/') if f.endswith(".txt") or f.endswith(".csv")]
# for f in filelist:
#     os.remove(os.path.join('/content/', f))
##sys.modules[__name__].__dict__.clear()

In [ ]:

import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import pandas as pd
# import csv, operator
import os
import sys
import scipy.stats as stats
import pylab as pl



data_path = '/content'
ds1_m0 = np.loadtxt(data_path +'/DS1_m_0.txt', delimiter = ',', usecols = range(20))
# print('DS1_m_0', ds1_m0)

ds1_m1 = np.loadtxt(data_path +'/DS1_m_1.txt', delimiter = ',', usecols = range(20))
# print('DS1_m_1', ds1_m1)

ds1_cov = np.loadtxt(data_path + '/DS1_Cov.txt', delimiter = ',', usecols = range(20))
# print('DS1_Cov is a', type(ds1_cov), ds1_cov.shape)

# plt.plot(ds1_cov)
# plt.title('DS1_Cov')
# fig = plt.gcf()
# fig.set_size_inches(10, 6)
# plt.show()

minus = np.random.multivariate_normal(ds1_m0, ds1_cov, 2000)
plus = np.random.multivariate_normal(ds1_m1, ds1_cov, 2000)
# fit = stats.norm.pdf(minus, np.mean(minus), np.std(minus))


# print('negative class is a', type(minus), minus.shape)
# print('positive class is a', type(plus), plus.shape)

minus = np.insert(minus, 20, 0, axis = 1)
plus = np.insert(plus, 20, 1, axis = 1)

# plt.hist(minus, label='negative')
# plt.ylim([0,1000])
# plt.legend(loc = 'upper right')
# plt.show()

# plt.hist(plus, label='positive')
# plt.legend(loc = 'upper right')
# plt.ylim([0,1000])
# plt.show()


train_minus = minus[800:]
valid_minus = minus[400:800]
test_minus = minus[:400]

train_plus = plus[800:]
valid_plus = plus[400:800]
test_plus = plus[:400]

ds1_train = np.append(train_minus , train_plus, axis=0)
ds1_valid = np.append(valid_minus , valid_plus, axis=0)
ds1_test = np.append(test_minus, test_plus, axis=0)

np.random.shuffle(ds1_train)
np.random.shuffle(ds1_valid)
np.random.shuffle(ds1_test)

DS1 = np.append(np.append(ds1_train, ds1_test, axis=0), ds1_valid, axis=0)

# print('train set is a', type(ds1_train), ds1_train.shape, 'like', ds1_train[0:1,0:4], '...')
# print('validation set is a', type(ds1_valid), ds1_valid.shape, 'like', ds1_valid[0:1,0:4], '...')
# print('test set is a', type(ds1_test), ds1_test.shape, 'like', ds1_test[0:1,0:4], '...')

# type(ds1_m0)
# print(ds1_m0)

np.savetxt('DS1_train.csv', ds1_train, delimiter=',')
np.savetxt('DS1_valid.csv', ds1_valid, delimiter=',')
np.savetxt('DS1_test.csv', ds1_test, delimiter=',')
np.savetxt('DS1.csv', DS1, delimiter=',')
# plt.plot(ds1_train)
# plt.polar(ds1_train)
# plt.plot(DS1)

> **2.** We first consider the $GDA$ model as seen in class: given the class variable, the data are
assumed to be Gaussians with different means for different classes but with the same
covariance matrix. This model can formally be specified as follows:

\begin{align*}
 Y \sim Bernoulli(\pi), && X|Y=j\sim \mathcal{N}(\mu_j, \Sigma).
\end{align*}

>>**2.1** Estimate the parameters of the $GDA$ model using the maximum likelihood approach.

>>>**2.1.a** For $DS1$, report the best fit accuracy, precision, recall and $F-$measure achieved by the classifier.

In [ ]:
print('Question #2.1.a', '\n')

from tabulate import tabulate


minus_num = 0
plus_num = 0
mu_0 = np.zeros(20)
mu_1= np.zeros(20)
s_0 = 0
s_1 = 0
n = len(ds1_train)
for row in ds1_train:
  if row[20] == 0:
    minus_num += 1
    mu_0 += row[:20]
  else:
    plus_num += 1
    mu_1 += row[:20]


p_0 = minus_num/(minus_num + plus_num)
p_1 = plus_num/(minus_num + plus_num)
mu_0 /= minus_num
mu_1/= plus_num


#---------------------------------------------------------

def cov_matrix(data_set):
  s_0 = 0
  s_1 = 0
  n = len(data_set)
  for row in data_set:
    if row[20]==0:
      last = np.array(row[:20])
      last -= mu_0
      last = np.reshape(last,(20,1))       
      s_0 += last.dot(last.T)

  s_0  /= minus_num
  for row in data_set:
    if row[20]==1:
      last = np.array(row[:20])
      last -= mu_1
      last = np.reshape(last,(20,1))    
      s_1 += last.dot(last.T)

  s_1 /= plus_num
  return p_0*s_0 + p_1*s_1
#---------------------------------------------------------

def gaussian_disc(data_set):
  cov_temp = np.linalg.inv(cov_matrix(data_set))
  w_1 = cov_temp.dot(mu_0 - mu_1)
  w_0 = -0.5*(mu_0.T).dot(cov_temp).dot(mu_0) + 0.5*(mu_1.T).dot(cov_temp).dot(mu_1) + np.log(minus_num/plus_num)
  print("w_1 =", '\n',w_1)
  print("w_0 =", '\n', w_0)
  return w_0, w_1
#---------------------------------------------------------

def activation(x, b, a):
  discriminant = b + a.dot(x)
  return 1/(1 + np.exp(-discriminant))
#---------------------------------------------------------

def GDA_params(data_set, b, a):
  true_positives = 0
  true_negatives = 0
  false_positives = 0
  false_negatives = 0
  for row in data_set:
    if row[-1] == 0:
      if activation(row[:-1], b, a) >= 0.5:
        true_negatives += 1
      else:
        false_positives += 1
    else:
      if activation(row[:-1], b, a) < 0.5:
        true_positives += 1
      else:
        false_negatives += 1
  # print(tabulate([['true_positives =', true_positives], ['false_positives =', false_positives],
  #                ['true_negatives =', true_negatives], ['false_negatives =', false_negatives]]))
  precision = true_positives/(true_positives + false_positives)
  recall = true_positives/(true_positives + false_negatives)
  accuracy = (true_positives + true_negatives)/(true_positives + false_negatives + true_negatives + false_positives)
  F_1 = 2*precision*recall/(precision + recall)

  # print and table setting
  # print(tabulate([['Accuracy =', accuracy], ['Precision =', precision],
  #                ['Recall =', recall], ['F_1 measure =', F_1]]))
  # matrix = np.array([[true_positives, false_positives],
  #                          [false_negatives, true_negatives]])
  # fig, ax = plt.subplots()
  # im = ax.imshow(matrix)
  # actual_value = ["Actual Positive", "Actual Negative"]
  # predicted_value = ["predicted Positive", "predicted Negative"]
  # ax.set_xticks(np.arange(len(predicted_value)))
  # ax.set_yticks(np.arange(len(actual_value)))
  # ax.set_xticklabels(predicted_value)
  # ax.set_yticklabels(actual_value)
  # plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
  # for i in range(len(actual_value)):
  #   for j in range(len(predicted_value)):
  #     text = ax.text(j, i, matrix[i, j],ha="center", va="center", color="r")
  # fig.tight_layout()
  # fig.show()
  np.savetxt("#2_1_a DS1 parameters.txt", ['DS1 parameters generated by GDA are: \n w_1 = {} \n'.format(w_1),
                                           'and \n w_0 = {} \n.'.format(w_0),
                                           'Other related values are:\n',
                                           'true_positives = {}'.format(true_positives),
                                           'true_negatives = {}'.format(true_negatives), 
                                           'false_positives = {}'.format(false_positives),
                                           'false_negatives = {} \n'.format(false_negatives),
                                           'F_1 measure = {}'.format(F_1),
                                           'Accuracy = {}'.format(accuracy),
                                           'Precision = {}'.format(precision),
                                           'Recall = {}'.format(recall)], fmt='%s')


w_0, w_1 = gaussian_disc(ds1_train)
#------------------------------------------------------------------------------
# parameters for test set
if __name__ == "__main__":
  GDA_params(ds1_test, w_0, w_1)


>>>**2.1.b** Report the coefficients learnt.

In [ ]:
print('Question #2.1.b', '\n')

def gaussian_disc(data_set):
  cov_temp = np.linalg.inv(cov_matrix(data_set))
  w_1 = cov_temp.dot(mu_0 - mu_1)
  w_0 = -0.5*(mu_0.T).dot(cov_temp).dot(mu_0) + 0.5*(mu_1.T).dot(cov_temp).dot(mu_1) + np.log(minus_num/plus_num)
  print('Coefficients are: \n', "w_1 =", '\n',w_1)
  print("w_0 =", '\n', w_0)
  return w_0, w_1


w_0, w_1 = gaussian_disc(ds1_train)
np.savetxt("#2_1_b coefficients learnt.txt", ['Coefficients are: \n w_0 = {},'.format(w_0), 
                                              ' and \n w_1 = {}.'.format(w_1)], fmt='%s')

>**3** For $DS1$, use $k-NN$ to learn a classifier. Repeat the experiment for different values of $k$
and report the performance for each value. We will compare this non-linear classifier to
the linear approach, and find out how powerful linear classifiers can be.

>>**3.a** Does this classifier performs better than $GDA$ or worse? Are there particular values
of $k$ which perform better? Why does this happen ? Use $F1-$Measure for model
selection.

>>**3.b** Report the best fit accuracy, precision, recall and $f-$measure achieved by this classifier.

In [ ]:
def KNN(x, k, data_set):
  n_neighbours = []
  n = len(data_set)
  distance = []
  in_or_out = 0
  for pow,row in enumerate(data_set):
    distance = np.linalg.norm(x - row[:20])
    if len(n_neighbours) < k:
      n_neighbours.append([distance , pow])
    else:
      n_neighbours = sorted(n_neighbours, key = lambda x: x[0])
      if distance < n_neighbours[k-1][0]:
        n_neighbours[-1] = [distance,pow]
  for neighbour in n_neighbours:
    in_or_out += data_set[neighbour[1]][-1]
  in_or_out /=k
  return in_or_out


def KNN_GDA_params(k, data_set):
  true_positives = 0
  true_negatives = 0
  false_positives = 0
  false_negatives = 0
  for row in data_set:
    if row[-1] == 0:
      if KNN(row[:-1],k, data_set) <= 0.5:
        true_negatives += 1
      else:
        false_positives += 1
    else:
      if KNN(row[:-1],k, data_set) > 0.5:
        true_positives += 1
      else:
        false_negatives += 1
  # print(tabulate([['KNN-GDA for k=', k], ['true_positives =', true_positives], ['false_positives =', false_positives],
  #               ['true_negatives =', true_negatives], ['false_negatives =', false_negatives]]))
  precision = true_positives/(true_positives + false_positives)
  recall = true_positives/(true_positives + false_negatives)
  accuracy = (true_positives + true_negatives)/(true_positives + false_negatives + true_negatives + false_positives)
  F_1 = 2*precision*recall/(precision + recall)
  params = [precision, recall, accuracy, F_1, k]
  # # print and table setting
  # '''
  # https://matplotlib.org/3.3.2/gallery/images_contours_and_fields/
  # image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py
  # '''
  # print(tabulate([['k =', k], ['Accuracy =', accuracy], ['Precision =', precision],
  #               ['Recall =', recall], ['F_1 measure =', F_1]]))
  # matrix = np.array([[true_positives, false_positives],
  #                         [false_negatives, true_negatives]])
  # fig, ax = plt.subplots()
  # im = ax.imshow(matrix)
  # actual_value = ["Actual Positive", "Actual Negative"]
  # predicted_value = ["predicted Positive", "predicted Negative"]
  # ax.set_xticks(np.arange(len(predicted_value)))
  # ax.set_yticks(np.arange(len(actual_value)))
  # ax.set_xticklabels(predicted_value)
  # ax.set_yticklabels(actual_value)
  # ax.set_title("KNN-GDA for k = {}".format(k))
  # plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
  # for i in range(len(actual_value)):
  #   for j in range(len(predicted_value)):
  #     text = ax.text(j, i, matrix[i, j],ha="center", va="center", color="w")
  # fig.tight_layout()
  # plt.show()
  with open('#3_a KNN-GDA parameters.txt','a') as f:
    np.savetxt(f, ['KNN_GDA for k={}'.format(k), 
                                                'true_positives = {}'.format(k, true_positives),
                                                'true_negatives = {}'.format(true_negatives), 
                                                'false_positives = {}'.format(false_positives),
                                                'false_negatives = {}'.format(false_negatives),
                                                'F_1 measure = {}'.format(F_1),
                                                'Accuracy = {}'.format(accuracy),
                                                'Precision = {}'.format(precision),
                                                'Recall = {}'.format(recall), '#'*40], fmt='%s')
  return params

In [ ]:
def bests(data_set):
  k_list = [3, 4, 5, 10, 11, 12, 15, 20, 25, 30, 35, 50, 70, 100]
  all = []
  for k in k_list:
    best_accuracy = []
    best_recall = []
    best_precision = []
    best_Fmeasure = []
    all.append(KNN_GDA_params(k, data_set))
  best_accuracy = max(all, key=lambda x: x[2])
  best_recall = max(all, key=lambda x: x[1])
  best_precision = max(all, key=lambda x: x[0])
  best_Fmeasure = max(all, key=lambda x: x[3])
  best_performance_overall = max(all, key=lambda x: [x[i] for i in range(3)])
  # print('best_accuracy {} is recorded for k = {}'.format(best_accuracy[2], best_accuracy[4]))
  # print('best_recall {} is recorded for k = {}'.format(best_recall[1], best_recall[4]))
  # print('best_precision {} is recorded for k = {}'.format(best_precision[0], best_precision[4]))
  # print('best_Fmeasure {} is recorded for k = {}'.format(best_Fmeasure[3], best_Fmeasure[4]))
  # print('best overall performance {} is recorded for k = {}'.format(best_performance_overall, best_performance_overall[4]))
  if data_set[0][20] == ds1_test[0][20]:
    np.savetxt("#3_b KNN-GDA bests.txt", ['KNN_GDA parameters for all k in the \n list = {} \n are \n {}'.format(k_list, all),
                                          'list of all parameters is \n {}'.format(all),
                                          'best_accuracy {} is recorded for k = {}'.format(best_accuracy[2], best_accuracy[4]),
                                          'best_recall {} is recorded for k = {}'.format(best_recall[1], best_recall[4]), 
                                          'best_precision {} is recorded for k = {}'.format(best_precision[0], best_precision[4]),
                                          'best_Fmeasure {} is recorded for k = {}\n'.format(best_Fmeasure[3], best_Fmeasure[4]),
                                          'best overall performance {} is recorded for k = {}'.format(best_performance_overall, 
                                                                                                      best_performance_overall[4]),
                                          '#'*40],fmt='%s')
  else:
    np.savetxt("#5_3 KNN-GDA D2 bests.txt", ['KNN_GDA parameters for all k in the \n list = {} \n are \n {}'.format(k_list, all),
                                             'list of all parameters is \n {}'.format(all),
                                             'best_accuracy {} is recorded for k = {}'.format(best_accuracy[2], best_accuracy[4]),
                                             'best_recall {} is recorded for k = {}'.format(best_recall[1], best_recall[4]), 
                                             'best_precision {} is recorded for k = {}'.format(best_precision[0], best_precision[4]),
                                             'best_Fmeasure {} is recorded for k = {}\n'.format(best_Fmeasure[3], best_Fmeasure[4]),
                                             'best overall performance {} is recorded for k = {}'.format(best_performance_overall, 
                                                                                                         best_performance_overall[4]),
                                             '#'*40],fmt='%s')
  return all 


In [ ]:
bests(ds1_test)
# print(all)

In [ ]:
# # visualization


# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator



# lables = ['precision', 'recall', 'accuracy', 'F_1', 'k']
# x = [3, 4, 5, 10, 11, 12, 15, 20, 25, 30, 35, 50, 70, 100]
# # l_list = all
# l_list = [[0.7897435897435897, 0.77, 0.7825, 0.7797468354430381, 3],
#           [0.8185185185185185, 0.5525, 0.715, 0.6597014925373134, 4],
#           [0.7240506329113924, 0.715, 0.72125, 0.7194968553459119, 5],
#           [0.694006309148265, 0.55, 0.65375, 0.6136680613668062, 10],
#           [0.6317073170731707, 0.6475, 0.635, 0.6395061728395062, 11],
#           [0.6676557863501483, 0.5625, 0.64125, 0.6105834464043418, 12],
#           [0.6428571428571429, 0.675, 0.65, 0.6585365853658537, 15],
#           [0.6556473829201102, 0.595, 0.64125, 0.6238532110091742, 20],
#           [0.6153846153846154, 0.64, 0.62, 0.6274509803921569, 25],
#           [0.6297297297297297, 0.5825, 0.62, 0.6051948051948052, 30],
#           [0.616822429906542, 0.66, 0.625, 0.6376811594202898, 35],
#           [0.6120218579234973, 0.56, 0.6025, 0.5848563968668408, 50],
#           [0.5882352941176471, 0.55, 0.5825, 0.5684754521963825, 70],
#           [0.5892351274787535, 0.52, 0.57875, 0.5524568393094289, 100]]

# for i in [x for x in range(len(l_list[0]))]:
#     plt.plot(x,[pt[i] for pt in l_list],label = '{}'.format(lables[i]))
# print('parameters performance with respect to the k')
# plt.legend()
# plt.axis([3,60,0.4, 0.85])
# plt.gcf().set_size_inches(15, 8)
# plt.xlabel("k")
# plt.show()

# # for i in range(len(l_list[0])):
# plt.plot(x,[pt[3] for pt in l_list],label = '%s'%lables[3])
# plt.gcf().set_size_inches(15, 8)
# plt.legend()
# plt.axis([3,60,0.45, 0.8])
# plt.xlabel("k")
# plt.show()

> **4.** Now instead of having a single multivariate Gaussian distribution per class, each class
is going to be generated by a mixture of $3$ Gaussians. For each class, we’ll define
$3$ Gaussians, with the first Gaussian of the first class sharing the covariance matrix
with the first Gaussian of the second class and so on. For both the classes, fix the
mixture probability as $(0.1,0.42,0.48)$ i.e. the sample has arisen from first Gaussian with
probability $0.1$, second with probability $0.42$ and so on. Mean for three Gaussians in the
positive class are given as $DS2-c1-m1$, $DS2-c1-m2, DS2-c1-m3$. Mean for three Gaussians
in the negative class are gives as $DS2-c2-m1$, $DS2-c2-m2$, $DS2-c2-m3$. Corresponding $3$
covariance matrices are given as $DS2-cov-1$, $DS2-cov-2$ and $DS2-cov-3$. Now sample
from this distribution and generate the dataset similar to question $1$. Call this dataset
as $DS2$, and submit it with your code. Follow the instructions from Assignment $1$ for
data submission format.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


data_path = '/content'

ds2_c1_m1 = np.loadtxt(data_path + "/DS2_c1_m1.txt", delimiter = ',', usecols=range(20))
ds2_c1_m2 = np.loadtxt(data_path + "/DS2_c1_m2.txt", delimiter = ',', usecols=range(20))
ds2_c1_m3 = np.loadtxt(data_path + "/DS2_c1_m3.txt", delimiter = ',', usecols=range(20))

ds2_c2_m1 = np.loadtxt(data_path + "/DS2_c2_m1.txt", delimiter = ',', usecols=range(20))
ds2_c2_m2 = np.loadtxt(data_path + "/DS2_c2_m2.txt", delimiter = ',', usecols=range(20))
ds2_c2_m3 = np.loadtxt(data_path + "/DS2_c2_m3.txt", delimiter = ',', usecols=range(20))

ds2_cov1 = np.loadtxt(data_path + "/DS2_Cov1.txt", delimiter = ',', usecols= range(20))
ds2_cov2 = np.loadtxt(data_path + "/DS2_Cov2.txt", delimiter = ',', usecols= range(20))
ds2_cov3 = np.loadtxt(data_path + "/DS2_Cov3.txt", delimiter = ',', usecols= range(20))

# plt.plot(ds2_cov1)
# plt.title('ds2_cov1')
# fig = plt.gcf()
# fig.set_size_inches(10, 6)
# plt.show()

# similar to the question #1
Probs_raisen = [0.10, 0.42, 0.48]
sample_norm = 2000
np.random.seed(1)
class_plus = []
class_minus = []
for i in range(2000):
  by_chance = np.random.random()
  if (by_chance <= 0.10):
    class_plus.append(np.random.multivariate_normal(ds2_c1_m1, ds2_cov1))
  elif (by_chance < 0.52):
    class_plus.append(np.random.multivariate_normal(ds2_c1_m2, ds2_cov2))
  else:
    class_plus.append(np.random.multivariate_normal(ds2_c1_m3, ds2_cov3))

for j in range(2000):
  by_chance = np.random.random()
  if (by_chance <= 0.10):
    class_minus.append(np.random.multivariate_normal(ds2_c2_m1, ds2_cov1))
  elif (by_chance < 0.52):
    class_minus.append(np.random.multivariate_normal(ds2_c1_m2, ds2_cov2))
  else:
    class_minus.append(np.random.multivariate_normal(ds2_c1_m3, ds2_cov3))


class_minus = np.insert(class_minus, 20, 0, axis = 1)
class_plus = np.insert(class_plus, 20, 1, axis = 1)

all_data = np.concatenate((class_plus, class_minus), axis = 0)
np.random.shuffle(all_data)

# plt.hist(class_minus, label='negative')
# plt.ylim([0,1000])
# plt.legend(loc = 'upper right')
# plt.show()

# plt.hist(class_plus, label='positive')
# plt.ylim([0,1000])
# plt.legend(loc = 'upper right')
# plt.show()

ds2_train = np.append(class_minus[800:] , class_plus[800:], axis=0)
ds2_valid = np.append(class_minus[400:800] , class_plus[400:800], axis=0)
ds2_test = np.append(class_minus[:400], class_plus[:400], axis=0)

np.random.shuffle(ds2_train)
np.random.shuffle(ds2_valid)
np.random.shuffle(ds2_test)

DS2 = np.append(np.append(ds2_train, ds2_test, axis=0), ds2_valid, axis=0)

# print('train set is a', type(ds2_train), ds2_train.shape, 'like', ds2_train[0:1,0:4], '...')
# print('validation set is a', type(ds2_valid), ds2_valid.shape, 'like', ds2_valid[0:1,0:4], '...')
# print('test set is a', type(ds2_test), ds2_test.shape, 'like', ds2_test[0:1,0:4], '...')

np.savetxt('DS2_train.csv', ds2_train, delimiter=',')
np.savetxt('DS2_valid.csv', ds2_valid, delimiter=',')
np.savetxt('DS2_test.csv', ds2_test, delimiter=',')
np.savetxt('DS2.csv', DS2, delimiter=',')


>**5.** Now perform the experiments in questions $2$ and $3$ again, but now using $DS2$.
>> **5.1.** Estimate the parameters of the $GDA$ model using the maximum likelihood ap-
proach.
>>>**5.1.a.** For $DS1$, report the best fit accuracy, precision, recall and F-measure achieved
by the classifier.

In [ ]:
minus_num_2 = 0
plus_num_2 = 0
mu_0_2 = np.zeros(20)
mu_1_2= np.zeros(20)
s_0_2 = 0
s_1_2 = 0
n_2 = len(ds2_train)
for row in ds2_train:
  if row[20] == 0:
    minus_num_2 += 1
    mu_0_2 += row[:20]
  else:
    plus_num_2 += 1
    mu_1_2 += row[:20]

p_0_2 = minus_num_2/n_2
p_1_2 = plus_num_2/n_2
mu_0_2 /= minus_num_2
mu_1_2/= plus_num_2

#---------------------------------------------------------

def cov_matrix(data_set):
  s_0_2 = 0
  s_1_2 = 0
  n_2 = len(data_set)
  for row in data_set:
    if row[20]==0:
      last_2 = np.array(row[:20])
      last_2 -= mu_0_2
      last_2 = np.reshape(last_2,(20,1))       
      s_0_2 += last_2.dot(last_2.T)

  s_0_2  /= minus_num_2
  for row in data_set:
    if row[20]==1:
      last_2 = np.array(row[:20])
      last_2 -= mu_1_2
      last_2 = np.reshape(last_2,(20,1))    
      s_1_2 += last_2.dot(last_2.T)

  s_1_2 /= plus_num_2
  return p_0_2*s_0_2 + p_1_2*s_1_2
#---------------------------------------------------------

def gaussian_disc(data_set):
  cov_temp = np.linalg.inv(cov_matrix(data_set))
  w_1_2 = cov_temp.dot(mu_0_2 - mu_1_2)
  w_0_2 = -.5*(mu_0_2.T).dot(cov_temp).dot(mu_0_2) + 0.5*(mu_1_2.T).dot(cov_temp).dot(mu_1_2) + np.log(minus_num_2/plus_num_2)
  print("w_1_2 =", '\n',w_1_2)
  print("w_0_2 =", '\n', w_0_2)
  return w_0_2, w_1_2
#---------------------------------------------------------

def activation(x, b, a):
  discriminant = b + a.dot(x)
  return 1/(1 + np.exp(-discriminant))
#---------------------------------------------------------

def GDA_params_2(data_set, b, a):
  true_positives_2 = 0
  true_negatives_2 = 0
  false_positives_2 = 0
  false_negatives_2 = 0
  for row in data_set:
    if row[-1] == 0:
      if activation(row[:-1], b, a) >= 0.5:
        true_negatives_2 += 1
      else:
        false_positives_2 += 1
    else:
      if activation(row[:-1], b, a) < 0.5:
        true_positives_2 += 1
      else:
        false_negatives_2 += 1
  # print(tabulate([['true_positives_2 =', true_positives_2], ['false_positives_2 =', false_positives_2],
  #                ['true_negatives_2 =', true_negatives_2], ['false_negatives_2 =', false_negatives_2]]))
  precision_2 = true_positives_2/(true_positives_2 + false_positives_2)
  recall_2 = true_positives_2/(true_positives_2 + false_negatives_2)
  accuracy_2 = (true_positives_2 + true_negatives_2)/(true_positives_2 + false_negatives_2 + 
                                                      true_negatives_2 + false_positives_2)
  F_1_2 = 2*precision_2*recall_2/(precision_2 + recall_2)

  # print and table setting
  # print(tabulate([['Accuracy_2 =', accuracy_2], ['Precision_2 =', precision_2],
  #                ['Recall_2 =', recall_2], ['F_1_2 measure =', F_1_2]]))
  # matrix_2 = np.array([[true_positives_2, false_positives_2],
  #                          [false_negatives_2, true_negatives_2]])
  # fig, ax = plt.subplots()
  # im = ax.imshow(matrix_2)
  # actual_value = ["Actual Positive", "Actual Negative"]
  # predicted_value = ["predicted Positive", "predicted Negative"]
  # ax.set_xticks(np.arange(len(predicted_value)))
  # ax.set_yticks(np.arange(len(actual_value)))
  # ax.set_xticklabels(predicted_value)
  # ax.set_yticklabels(actual_value)
  # plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
  # for i in range(len(actual_value)):
  #   for j in range(len(predicted_value)):
  #     text = ax.text(j, i, matrix_2[i, j],ha="center", va="center", color="r")
  # fig.tight_layout()
  # fig.show()
  np.savetxt("#5_1_a DS2 parameters.txt", ['DS2 other variables are \n ',
                                           'true_positives_2 = {}'.format(true_positives_2),
                                            'true_negatives_2 = {}'.format(true_negatives_2), 
                                            'false_positives_2 = {}'.format(false_positives_2),
                                            'false_negatives_2 = {}'.format(false_negatives_2),
                                            'F_1_2 measure = {}'.format(F_1_2),
                                            'Accuracy_2 = {}'.format(accuracy_2),
                                            'Precision_2 = {}'.format(precision_2),
                                            'Recall_2 = {}'.format(recall_2)], fmt='%s')


w_0_2, w_1_2 = gaussian_disc(ds2_train)
#------------------------------------------------------------------------------
# parameters for test set

GDA_params_2(ds2_test, w_0_2, w_1_2)

>>>**5.1.b.** Report the coefficients learnt.

In [ ]:
w_0_2, w_1_2 = gaussian_disc(ds2_train)
np.savetxt("#5_1_b coefficients.txt", ['coefficients are \n w_0_2 = {}'.format(w_0_2), 'w_1_2 = {}'.format(w_1_2)], fmt='%s')

>>**5.2.** Does $k-NN$ classifier perform better than $GDA$ or worse? Are there particular
values of $k$ which perform better? Why does this happen ?

>>**5.3.** Report the best fit accuracy, precision, recall and $f-$measure achieved by this clas-
sifier.

In [ ]:

bests(ds2_test)


In [ ]:
# # visualization

# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator


# lables = ['precision_2', 'recall_2', 'accuracy_2', 'F_1_2', 'k']
# x = [3, 4, 5, 10, 11, 12, 15, 20, 25, 30, 35, 50, 70, 100]
# # l_list = all
# l_list = [[0.7524038461538461, 0.7825, 0.7625, 0.767156862745098, 3],
#           [0.8214285714285714, 0.5175, 0.7025, 0.6349693251533742, 4],
#           [0.6926829268292682, 0.71, 0.6975, 0.7012345679012345, 5],
#           [0.6794425087108014, 0.4875, 0.62875, 0.5676855895196506, 10],
#           [0.6085858585858586, 0.6025, 0.6075, 0.6055276381909548, 11],
#           [0.6462585034013606, 0.475, 0.6075, 0.547550432276657, 12],
#           [0.6122448979591837, 0.6, 0.61, 0.6060606060606062, 15],
#           [0.625, 0.5, 0.6, 0.5555555555555556, 20],
#           [0.5738498789346247, 0.5925, 0.57625, 0.5830258302583026, 25],
#           [0.5816023738872403, 0.49, 0.56875, 0.5318860244233379, 30],
#           [0.5564102564102564, 0.5425, 0.555, 0.549367088607595, 35],
#           [0.5447154471544715, 0.5025, 0.54125, 0.5227568270481144, 50],
#           [0.5135135135135135, 0.5225, 0.51375, 0.5179677819083023, 70],
#           [0.5114155251141552, 0.56, 0.5125, 0.5346062052505968, 100]]
# for i in [x for x in range(len(l_list[0]))]:
#     plt.plot(x,[pt[i] for pt in l_list],label = '{}'.format(lables[i]))
# print('parameters performance with respect to the k')
# plt.legend()
# plt.axis([3,65,0.2, 1])
# plt.gcf().set_size_inches(15, 8)
# plt.xlabel("k")
# plt.show()

# # for i in range(len(l_list[0])):
# plt.plot(x,[pt[3] for pt in l_list],label = '%s'%lables[3])
# plt.gcf().set_size_inches(15, 8)
# plt.legend()
# plt.axis([3,60,0.3, 1])
# plt.xlabel("k")
# plt.show()

>**6.** Comment on any similarities and differences between the performance of both classifiers
on datasets DS1 and $DS2$?